In [21]:
import numpy as np
import pandas as pd

In [22]:
s1415 = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\Premier League\Season 2014-2015.csv')

understats = pd.read_csv(r'C:\Users\George\Desktop\football-match-prediction-using-ml-techniques\data\understat_per_game.csv')

In [23]:
s1415 = s1415[['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG','HTHG','HTAG', 'FTR',
              'HS', 'AS', 'HST','AST','HC','AC', 'HF', 'AF', 'B365H', 'B365A', 'B365D',
               'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA']].reset_index(drop=True)

In [24]:
s1415.shape

(381, 28)

In [25]:
s1415.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,FTR,HS,AS,...,B365D,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA
0,16/08/14,Arsenal,Crystal Palace,2.0,1.0,1.0,1.0,H,14.0,4.0,...,6.5,1.26,6.45,14.01,1.25,5.5,12.0,1.25,6.25,10.50
1,16/08/14,Leicester,Everton,2.0,2.0,1.0,2.0,D,11.0,13.0,...,3.4,3.14,3.38,2.46,3.10,3.1,2.4,3.20,3.40,2.40
2,16/08/14,Man United,Swansea,1.0,2.0,0.0,1.0,A,14.0,5.0,...,5.0,1.37,5.10,10.60,1.36,4.5,9.0,1.36,5.20,10.00
3,16/08/14,QPR,Hull,0.0,1.0,0.0,0.0,A,19.0,11.0,...,3.3,2.48,3.26,3.22,2.60,3.0,2.9,2.55,3.20,3.12
4,16/08/14,Stoke,Aston Villa,0.0,1.0,0.0,0.0,A,12.0,7.0,...,3.5,1.95,3.47,4.55,1.95,3.2,4.2,1.95,3.30,4.75


In [26]:
s1415['Date'] = pd.to_datetime(s1415['Date'],format="%d/%m/%y")

In [27]:
understats['Date'] = pd.to_datetime(understats['date']).dt.strftime("%Y-%m-%d")
understats['Date'] = pd.to_datetime(understats['Date'])

In [28]:
understats.team.unique()

array(['Bayern Munich', 'Hamburger SV', 'Bayer Leverkusen', 'Hoffenheim',
       'Augsburg', 'Hertha Berlin', 'Werder Bremen', 'Schalke 04',
       'Mainz 05', 'Hannover 96', 'Borussia Dortmund',
       'Borussia M.Gladbach', 'Wolfsburg', 'Eintracht Frankfurt',
       'VfB Stuttgart', 'FC Cologne', 'Freiburg', 'Paderborn',
       'Ingolstadt', 'Darmstadt', 'RasenBallsport Leipzig',
       'Fortuna Duesseldorf', 'Nuernberg', 'Union Berlin', 'Aston Villa',
       'Everton', 'Southampton', 'Leicester', 'West Bromwich Albion',
       'Sunderland', 'Crystal Palace', 'Chelsea', 'West Ham', 'Tottenham',
       'Arsenal', 'Swansea', 'Stoke', 'Newcastle United', 'Liverpool',
       'Manchester City', 'Manchester United', 'Hull', 'Burnley',
       'Queens Park Rangers', 'Bournemouth', 'Norwich', 'Watford',
       'Middlesbrough', 'Huddersfield', 'Brighton', 'Cardiff', 'Fulham',
       'Wolverhampton Wanderers', 'Sheffield United', 'Malaga', 'Sevilla',
       'Deportivo La Coruna', 'Real Sociedad

In [29]:
s1415['HomeTeam'].unique()

array(['Arsenal', 'Leicester', 'Man United', 'QPR', 'Stoke', 'West Brom',
       'West Ham', 'Liverpool', 'Newcastle', 'Burnley', 'Aston Villa',
       'Chelsea', 'Crystal Palace', 'Everton', 'Southampton', 'Swansea',
       'Hull', 'Sunderland', 'Tottenham', 'Man City', nan], dtype=object)

In [30]:
understats =understats.replace({'team':{'Aston Villa':'Aston Villa',
       'Everton':'Everton', 'Southampton':'Southampton', 'Leicester':'Leicester', 'West Bromwich Albion':'West Brom',
       'Sunderland':'Sunderland', 'Crystal Palace':'Crystal Palace', 'Chelsea':'Chelsea', 'West Ham':'West Ham', 'Tottenham':'Tottenham',
       'Arsenal':'Arsenal', 'Swansea':'Swansea', 'Stoke':'Stoke', 'Newcastle United':'Newcastle', 'Liverpool':'Liverpool',
       'Manchester City':'Man City', 'Manchester United':'Man United', 'Hull':'Hull', 'Burnley':'Burnley',
       'Queens Park Rangers':'QPR', 'Bournemouth':'Bournemouth', 'Norwich':'Norwich', 'Watford':'Watford',
       'Middlesbrough':'Middlesbrough', 'Huddersfield':'Huddersfield', 'Brighton':'Brighton', 'Cardiff':'Cardiff', 'Fulham':'Fulham',
       'Wolverhampton Wanderers':'Wolves'}})

In [31]:
df = pd.merge(s1415,understats, how='inner',left_on=['Date','HomeTeam'],right_on=['Date','team'])

In [32]:
df.shape

(380, 57)

In [33]:
df.rename(columns = {'xG':"HxG", 'xGA':'AxG','deep':'Hdeep','deep_allowed':'Adeep',
                     'ppda_coef':'Hppda_coef','oppda_coef':'Appda_coef'},inplace=True)

df = df.drop(['npxG','npxGA','scored', 'missed', 'xpts', 'result', 'wins',
              'draws', 'loses', 'pts', 'npxGD', 'ppda_att', 'ppda_def',
              'oppda_att', 'oppda_def', 'team', 'xG_diff', 'xGA_diff',
              'xpts_diff', 'date', 'year','h_a'],axis=1)

In [34]:
# Calculating the average of betting sites odd for each result
avgH = round((df['B365H'] + df['PSH'] + df['WHH'] + df['VCH'])/4,2)
avgD = round((df['B365D'] + df['PSD'] + df['WHD'] + df['VCD'])/4,2)
avgA = round((df['B365A'] + df['PSA'] + df['WHA'] + df['VCA'])/4,2)

df['AVGH'] = avgH
df['AVGD'] = avgD
df['AVGA'] = avgA

#drop the existing columns with betting odds
df = df.drop(['PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 
              'VCD', 'VCA', 'B365H', 'B365D', 'B365A'],axis=1)

In [35]:
df['season'] = '2014/2015'

In [36]:
df.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,FTR,HS,AS,...,HxG,AxG,Hdeep,Adeep,Hppda_coef,Appda_coef,AVGH,AVGD,AVGA,season
0,2014-08-16,Arsenal,Crystal Palace,2.0,1.0,1.0,1.0,H,14.0,4.0,...,1.554110,0.158151,9,0,2.933333,14.172414,1.25,6.18,12.88,2014/2015
1,2014-08-16,Leicester,Everton,2.0,2.0,1.0,2.0,D,11.0,13.0,...,1.278300,0.613273,1,15,14.480000,11.687500,3.16,3.32,2.42,2014/2015
2,2014-08-16,Man United,Swansea,1.0,2.0,0.0,1.0,A,14.0,5.0,...,1.166350,0.278076,8,2,10.120000,17.947368,1.36,4.95,10.15,2014/2015
3,2014-08-16,QPR,Hull,0.0,1.0,0.0,0.0,A,19.0,11.0,...,1.900670,1.117570,1,5,12.823529,10.285714,2.53,3.19,3.08,2014/2015
4,2014-08-16,Stoke,Aston Villa,0.0,1.0,0.0,0.0,A,12.0,7.0,...,0.423368,0.909774,3,4,4.125000,14.043478,1.95,3.37,4.50,2014/2015


In [37]:
df.head(10)

,Date,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,FTR,HS,AS,...,HxG,AxG,Hdeep,Adeep,Hppda_coef,Appda_coef,AVGH,AVGD,AVGA,season
0,2014-08-16,Arsenal,Crystal Palace,2.0,1.0,1.0,1.0,H,14.0,4.0,...,1.554110,0.158151,9,0,2.933333,14.172414,1.25,6.18,12.88,2014/2015
1,2014-08-16,Leicester,Everton,2.0,2.0,1.0,2.0,D,11.0,13.0,...,1.278300,0.613273,1,15,14.480000,11.687500,3.16,3.32,2.42,2014/2015
2,2014-08-16,Man United,Swansea,1.0,2.0,0.0,1.0,A,14.0,5.0,...,1.166350,0.278076,8,2,10.120000,17.947368,1.36,4.95,10.15,2014/2015
3,2014-08-16,QPR,Hull,0.0,1.0,0.0,0.0,A,19.0,11.0,...,1.900670,1.117570,1,5,12.823529,10.285714,2.53,3.19,3.08,2014/2015
4,2014-08-16,Stoke,Aston Villa,0.0,1.0,0.0,0.0,A,12.0,7.0,...,0.423368,0.909774,3,4,4.125000,14.043478,1.95,3.37,4.50,2014/2015
5,2014-08-16,West Brom,Sunderland,2.0,2.0,1.0,1.0,D,10.0,7.0,...,1.683430,0.991901,6,3,5.935484,13.687500,2.25,3.28,3.53,2014/2015
6,2014-08-16,West Ham,Tottenham,0.0,1.0,0.0,0.0,A,18.0,10.0,...,1.853100,1.017060,9,3,11.250000,10.956522,3.88,3.49,2.05,2014/2015
7,2014-08-17,Liverpool,Southampton,2.0,1.0,1.0,0.0,H,12.0,12.0,...,1.331920,1.552480,6,9,11.333333,11.138889,1.35,5.24,9.82,2014/2015
8,2014-08-17,Newcastle,Man City,0.0,2.0,0.0,1.0,A,12.0,13.0,...,0.965583,1.480530,6,9,15.590909,12.909091,5.39,4.12,1.66,2014/2015
9,2014-08-18,Burnley,Chelsea,1.0,3.0,1.0,3.0,A,9.0,11.0,...,0.371141,2.129960,2,10,17.952381,13.733333,9.44,4.70,1.40,2014/2015


In [38]:
df.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'FTR',
       'HS', 'AS', 'HST', 'AST', 'HC', 'AC', 'HF', 'AF', 'league', 'HxG',
       'AxG', 'Hdeep', 'Adeep', 'Hppda_coef', 'Appda_coef', 'AVGH', 'AVGD',
       'AVGA', 'season'],
      dtype='object')

In [39]:
df.to_csv('epl-1415.csv', index=False)